In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
# TTS_PATH = \"/home/erogol/projects/\""
!pip install librosa numpy pandas scipy tqdm matplotlib pydub 
!pip install speech2text torch trainer transformers datasets[audio]
!pip install TTS

import os
import sys
import librosa
import numpy as np
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from matplotlib import pylab as plt
from collections import Counter
from pydub import AudioSegment
# import wave
%matplotlib inline

import torch
import transformers
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig, BaseAudioConfig
from TTS.tts.datasets.formatters import *
from TTS.tts.datasets import load_tts_samples
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
from datasets import Dataset
from datasets import load_dataset


ERROR: Could not find a version that satisfies the requirement speech2text (from versions: 0.1.0, 0.2.0, 0.2.1, 0.3.0a1, 0.3.0a2, 0.3.0a3)
ERROR: No matching distribution found for speech2text


In [ ]:
# DEFINE FUNCTIONS
    
def segment_audio_file(audio_file, output_path, mean_length, std_deviation):
    
    # Load the audio segment using pydub
    audio_segment = AudioSegment.from_file(audio_file)
    
    audio_length = len(audio_segment)
    
    # Calculate the desired number of segments based on audio length
    num_segments = int(audio_length / mean_length)
    
    # Generate random lengths following a normal distribution
    random_lengths = np.random.normal(loc=mean_length, scale=std_deviation, size=num_segments)
    
    # Convert lengths to integers and ensure they are positive
    random_lengths = np.maximum(1, random_lengths.astype(int))
    
    # Split the audio segment into random segments
    segment_start = 0
    segments = []
    
    for length in random_lengths:
        segment_end = segment_start + length
        segment = audio_segment[segment_start:segment_end]

        # Get just the filename without the root path
        just_filename = os.path.basename(audio_file)
        
        segment.export(os.path.join(output_path, f"{just_filename}_segment_{len(segments)}.mp3"), format="mp3")
        segments.append(segment)
        segment_start = segment_end

    return segments

              
    
def segment_audio(audio_path, output_path, mean_length, std_deviation):
    
    # Create a directory for saving the segments
    os.makedirs(output_path, exist_ok=True)
    
    for root, _, files in os.walk(audio_path):
        for filename in files:
            audio_file = os.path.join(root, filename)
            segment_audio_file(audio_file, output_path, mean_length, std_deviation)
            
            
                                               
def speech_recognition(segmented_audio_path, csv_path):
    
    os.makedirs(segmented_audio_path, exist_ok=True)

    model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
    processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

    dataset = load_dataset("audiofolder", data_dir=segmented_audio_path)

    for ds in dataset:
        
        inputs = processor(ds["audio"]["array"], sampling_rate=ds["audio"]["sampling_rate"], return_tensors="pt")
        generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

        # Save the transcription as a text file with the same name as the audio file
        audio_file_name = os.path.basename(ds["file"])
        transcription_file_name = os.path.splitext(audio_file_name)[0] + ".txt"
        transcription_file_path = os.path.join(segmented_audio_path, transcription_file_name)

        with open(transcription_file_path, "w", encoding="utf-8") as f:
            f.write(transcription[0])  # Assuming you're dealing with a single transcription
            
        # Update: Append audio file name and transcription to metadata list
        audio_file_name = os.path.basename(ds["file"])
        metadata.append({"audio_file": audio_file_name, "transcription": transcription[0]})

    # Write metadata to a metadata.csv file with "|" delimiter
    metadata_file_path = os.path.join(csv_path, "metadata.csv")
    with open(metadata_file_path, "w", encoding="utf-8") as metadata_file:
        metadata_file.write("audio_file|transcription\n")  # Header
        for item in metadata:
            metadata_file.write(f"{item['audio_file']}|{item['transcription']}\n")
    
    
    
def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "my_speaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0])
            text = cols[1]
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items




# CONFIGURE DATA
    
# declare some things 
original_audio_path = "/kaggle/input/voicescarab/"
segmented_audio_path = "/kaggle/working/segmented_audio/wavs/"
csv_path = "/kaggle/working/segmented_audio/"
# output_path = os.path.dirname(os.path.abspath(__file__))
# only output is write-able
output_path = "/kaggle/working/"
mean_length = 30 * 1000  # Mean segment length in milliseconds (30 seconds)
std_deviation = 7 * 1000  # Standard deviation of the segment lengths (7 seconds)


# first segment audio and put it into a new directory
segment_audio(original_audio_path, segmented_audio_path, mean_length, std_deviation)
print('Segments created. \n')

# run speech recognition on each of the files in the segmented_audio directory
speech_recognition(segmented_audio_path, csv_path)
print('Transcriptions created. \n')

# dataset config for one of the pre-defined datasets
dataset_config = BaseDatasetConfig(
    formatter=formatter, meta_file_train="metadata.csv", language="en-us", path=csv_path
)

audio_config = BaseAudioConfig(
    sample_rate=22050,
    resample=False,  # Resample to 22050 Hz. It slows down training. Use `TTS/bin/resample.py` to pre-resample and set this False for faster training.
    do_trim_silence=True,
    trim_db=23.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    preemphasis=0.0,
)

print('Configured successfully, cell completed.')


In [ ]:
# ANALYZE DATASET

# use your own preprocessor at this stage - TTS/datasets/proprocess.py
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=formatter)
if eval_samples is not None:
    items = train_samples + eval_samples
else:
    items = train_samples
print(" > Number of audio files: {}".format(len(items)))
print(items[1])

# check wavs if exist
wav_files = []
for item in items:
    wav_file = item["audio_file"].strip()
    wav_files.append(wav_file)
    if not os.path.exists(wav_file):
        print(waf_path)

# show duplicate items
c = Counter(wav_files)
print([item for item, count in c.items() if count > 1])

item


def load_item(item):
    text = item["text"].strip()
    file_name = item["audio_file"].strip()
    audio, sr = librosa.load(file_name, sr=None)
    audio_len = len(audio) / sr
    text_len = len(text)
    return file_name, text, text_len, audio, audio_len

NUM_PROC = 8


# This will take a while depending on size of dataset
if NUM_PROC == 1:
    data = []
    for m in tqdm(items):
        data += [load_item(m)]
else:
    with Pool(8) as p:
        data = list(tqdm(p.imap(load_item, items), total=len(items)))

# count words in the dataset
w_count = Counter()
for item in tqdm(data):
    text = item[1].lower().strip()
    for word in text.split():
        w_count[word] += 1
print(" > Number of words: {}".format(len(w_count)))


text_vs_durs = {}  # text length vs audio duration
text_len_counter = Counter()  # number of sentences with the keyed length
for item in tqdm(data):
    text = item[1].lower().strip()
    text_len = len(text)
    text_len_counter[text_len] += 1
    audio_len = item[-1]
    try:
        text_vs_durs[text_len] += [audio_len]
    except:
        text_vs_durs[text_len] = [audio_len]


# text_len vs avg_audio_len, median_audio_len, std_audio_len
text_vs_avg = {}
text_vs_median = {}
text_vs_std = {}
for key, durs in text_vs_durs.items():
    text_vs_avg[key] = np.mean(durs)
    text_vs_median[key] = np.median(durs)
    text_vs_std[key] = np.std(durs)


for item in data:
    if item[-1] < 2:
        print(item)

sec_per_chars = []
for item in data:
    text = item[1]
    dur = item[-1]
    sec_per_char = dur / len(text)
    sec_per_chars.append(sec_per_char)
# sec_per_char /= len(data)
# print(sec_per_char)

mean = np.mean(sec_per_chars)
std = np.std(sec_per_chars)
print(mean)
print(std)

dist = norm(mean, std)

# find irregular instances long or short voice durations
for item in data:
    text = item[1]
    dur = item[-1]
    sec_per_char = dur / len(text)
    pdf =norm.pdf(sec_per_char)
    if pdf < 0.39:
        print(item)


plt.title("text length vs mean audio duration")
plt.scatter(list(text_vs_avg.keys()), list(text_vs_avg.values()))
plt.title("text length vs median audio duration")
plt.scatter(list(text_vs_median.keys()), list(text_vs_median.values()))
plt.title("text length vs STD")
plt.scatter(list(text_vs_std.keys()), list(text_vs_std.values()))
plt.title("text length vs # instances")
plt.scatter(list(text_len_counter.keys()), list(text_len_counter.values()))


w_count_df = pd.DataFrame.from_dict(w_count, orient='index')
w_count_df.sort_values(0, ascending=False, inplace=True)
w_count_df
# check a certain word
w_count_df.at['minute', 0]
# fequency bar plot - it takes time!!
w_count_df.plot.bar()

In [ ]:
# TRAIN MODEL

config = Tacotron2Config(  # This is the config that is saved for the future use
    audio=audio_config,
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=2,
    # gradual_training=[[0, 6, 48], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
    double_decoder_consistency=False,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=150,
    print_eval=False,
    mixed_precision=True,
    min_text_len=0,
    max_text_len=500,
    min_audio_len=0,
    max_audio_len=44000 * 10,
    output_path=output_path,
    datasets=[dataset_config],
    use_speaker_embedding=True,  # set this to enable multi-sepeaker training
    decoder_ssim_alpha=0.0,  # disable ssim losses that causes NaN for some runs.
    postnet_ssim_alpha=0.0,
    postnet_diff_spec_alpha=0.0,
    decoder_diff_spec_alpha=0.0,
    attention_norm="softmax",
    optimizer="Adam",
    lr_scheduler=None,
    lr=3e-5,
)

## INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init speaker manager for multi-speaker training
# it mainly handles speaker-id to speaker-name for the model and the data-loader
speaker_manager = SpeakerManager()
speaker_manager.set_ids_from_data(train_samples + eval_samples, parse_key="speaker_name")

# init model
model = Tacotron2(config, ap, tokenizer, speaker_manager)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

In [ ]:
# EXTRANEOUS NOTES

'''
first segment and save the mp3s in a new dir
then stt the mp3s and save txt files side by side
 - need to put them in audio_dataset format
 
 
 
[os.path.join(dirpath, filename) for dirpath, _, filenames in os.walk(wav_dir_path) for filename in filenames]

 
 
     
    audio_dataset = Dataset.from_dict({"audio": [os.path.join(dirpath, filename) for filename in os.walk(dirpath)]}).cast_column("audio", Audio())
    audio_dataset[0]["audio"]
    {'array': array([ 0.        ,  0.00024414, -0.00024414, ..., -0.00024414,
             0.        ,  0.        ], dtype=float32),
     'path': 'path/to/audio_1',
     'sampling_rate': 16000}

       
seg = audiosegment.from_file("some_audio.wav").resample(sample_rate_Hz=24000, sample_width=2, channels=1)
    


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# then get it in the dataset analysis, then in the format needed for training

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    txt_file = os.path.join(root_path, meta_file)
    items = []
    speaker_name = "myspeaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")            
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


def load_voice_scarab(wav_dir_path, all_base_wavs):

    for filenames in os.walk(wav_dir_path):
        for filename in filenames:
            with wave.open(os.path.join(wav_dir_path, filename), 'rb') as wav_file:
                num_channels = wav_file.getnchannels()
                sample_width = wav_file.getsampwidth()
                frame_rate = wav_file.getframerate()
                num_frames = wav_file.getnframes()
                audio_data = wav_file.readframes(num_frames)
                all_base_wavs.append({'num_channels': num_channels, 
                                  'sample_width': sample_width, 
                                  'frame_rate': frame_rate, 
                                  'num_frames': num_frames, 
                                  'audio_data': audio_data,
                                  'filename': filename})
       
    
    return all_base_wavs




def speech_recognition(segmented_audio_path):

    model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
    processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

    dataset = load_dataset("audiofolder", data_dir=segmented_audio_path)

    inputs = processor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="pt")
    generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)




def segment_audio(audio_path, output_path):
    
    # Create a directory for saving the segments
    os.makedirs(output_path, exist_ok=True)
    
    # Define the desired distribution parameters
    mean_length = 30 * 1000  # Mean segment length in milliseconds (30 seconds)
    std_deviation = 7 * 1000  # Standard deviation of the segment lengths (7 seconds)
    
    for filename in os.walk(audio_path):
        # segment that file

        # Load the audio segment using pydub
        audio_segment = AudioSegment.from_file(os.path.join(audio_path, filename)

        # Generate random lengths following a normal distribution
        num_segments = 10  # Number of segments
        random_lengths = np.random.normal(loc=mean_length, scale=std_deviation, size=num_segments)

        # Convert lengths to integers and ensure they are positive
        random_lengths = np.maximum(1, random_lengths.astype(int))

        # Split the audio segment into random segments
        segment_start = 0
        segments = []

        for length in random_lengths:
            segment_end = segment_start + length
            segment = audio_segment[segment_start:segment_end]
            segment.export(os.path.join(output_dir, f"segment_{len(segments)}.mp3"), format="mp3")
            segments.append(segment)
            segment_start = segment_end

                                               
    return segments

 
 

'''
